# Init & parsing


In [2]:
from collections import Counter
from application import *
from pre_processing import Job, Systems, Objects 
import statistics
import numpy as np
import matplotlib.pyplot as plt
from journal import Journal, JournalASU, jobs_to_df
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [24]:
jobs = []
jobs.extend(process_files(r'.\input data\1', all_visible_sheets, works_parser.ParserSake))
jobs.extend(process_files(r'.\input data\2', all_visible_sheets, works_parser.ParserSake)) # ASU
# jobs.extend(process_files(r'.\input data\АСУ', find_sheets_asu, works_parser.ParserAsu))
# jobs.extend(process_files(r'.\input data\ВОЛС', all_visible_sheets, works_parser.ParserVols))
# jobs.extend(process_files(r'.\input data\Телеканал', find_sheets_vols, works_parser.ParserTk))
# jobs.extend(process_files(r'.\input data\АИИСКУЭ', find_sheets_vols, works_parser.ParserAskueSake))
# jobs.extend(process_files(r'.\input data\АИИСКУЭ', find_sheets_vols, works_parser.ParserAskue))
# jobs.extend(process_files(r'.\input data\Тех.учет', find_sheets_vols, works_parser.ParserTechReg))
print(len(jobs))

In [25]:
stats = Counter((j.system, j.object)for j in jobs)    
# print(stats)

In [26]:
statistics.visualize_statistics(stats)

In [27]:
for (xlabel, ylabel), count in stats.items():
    print(f'{str(xlabel).ljust(17)}, {str(ylabel).ljust(15)} - {str(count).ljust(5)}')

# Tests

ASU test

In [7]:
assert stats[(Systems.ASU_TP, Objects.S1)] > 0
assert stats[(Systems.ASU_TP, Objects.S2)] > 0
assert stats[(Systems.ASU_TP, Objects.V1)] > 0
assert stats[(Systems.ASU_TP, Objects.V2)] > 0
assert stats[(Systems.ASU_TP, Objects.V3)] > 0
assert stats[(Systems.ASU_TP, Objects.V4)] > 0
assert stats[(Systems.ASU_TP, Objects.V5)] > 0
assert stats[(Systems.ASU_TP, Objects.V6)] > 0
assert stats[(Systems.ASU_TP, Objects.ZU)] > 0

assert stats[(Systems.ASU_I, Objects.S1)] > 0
assert stats[(Systems.ASU_I, Objects.S2)] > 0
assert stats[(Systems.ASU_I, Objects.ZU)] > 0

assert stats[(Systems.ASU_AM, Objects.S2)] > 0

assert stats[(Systems.LVS, Objects.S1)] > 0
assert stats[(Systems.LVS, Objects.S2)] > 0
assert stats[(Systems.LVS, Objects.V1)] > 0
assert stats[(Systems.LVS, Objects.V2)] > 0
assert stats[(Systems.LVS, Objects.V3)] > 0
assert stats[(Systems.LVS, Objects.V4)] > 0
assert stats[(Systems.LVS, Objects.V5)] > 0
assert stats[(Systems.LVS, Objects.V6)] > 0
assert stats[(Systems.LVS, Objects.ZU)] > 0

print('all ASU/LVS tests is complete')

VOLS test

In [9]:
assert stats[(Systems.VOLS, Objects.ZU)] > 0
assert stats[(Systems.VOLS, Objects.PS360)] > 0
assert stats[(Systems.VOLS, Objects.S2)] > 0
assert stats[(Systems.VOLS, Objects.S1)] > 0

print('all VOLS tests is complete')

TK test

In [10]:
assert stats[(Systems.TK, Objects.ZU)] > 0
assert stats[(Systems.TK, Objects.PS360)] > 0
assert stats[(Systems.TK, Objects.S2)] > 0
assert stats[(Systems.TK, Objects.S1)] > 0

print('all TK tests is complete')

ASKUE test

In [11]:
assert stats[(Systems.ASKUE, Objects.ZU)] > 0
assert stats[(Systems.ASKUE, Objects.PS360)] > 0
# assert stats[(Systems.ASKUE, Objects.PS223)] > 0
assert stats[(Systems.ASKUE, Objects.S2)] > 0
assert stats[(Systems.ASKUE, Objects.S1)] > 0

print('all ASKUE tests is complete')

TECH_REG test

In [12]:
assert stats[(Systems.TECH_REG, Objects.PS360)] > 0
assert stats[(Systems.TECH_REG, Objects.S2)] > 0
assert stats[(Systems.TECH_REG, Objects.S1)] > 0

print('all TECH_REG tests is complete')

# Sandbox

In [28]:
import pandas as pd
from pre_processing import Systems, Objects

In [29]:
df_jobs = jobs_to_df(jobs)

In [31]:
df_jobs.head()

In [79]:
from journal import Journal, JournalASU, JournalASKUE, jobs_to_df, batch_journal_generator
from config_journals import batch_ASU_journals, batch_ASKUE_journals

In [58]:
j1 = JournalASKUE(df_jobs)
j1.make_journal(Systems.TECH_REG)
j1.journal
# j1.save_journal('test')

In [54]:
batch_journal_generator(df_jobs, JournalASU, batch_ASU_journals)


In [80]:
batch_journal_generator(df_jobs, JournalASKUE, batch_ASKUE_journals)

In [129]:
columns = ('date', 'system', 'object', 'place', 'work_type', 'tech_map', 'equip_name', 'performer', )
result = pd.DataFrame(columns=columns,
                      data = ((j.date, j.system, j.object, j.place, j.work_type, 
                               j.tech_map, j.equip_name, j.performer) 
                              for j in jobs))

In [125]:
result = df_jobs

In [121]:
result[result.isna().any(axis=1)]


In [132]:
result[result.system == Systems.VOLS]\
    [result.object == Objects.S1]\
    .drop(['system', 'object'], axis=1)\
    .reindex(columns=['date', 'place', 'work_type', 'tech_map', 'performer'])\
    .sort_values(by=['date', 'place', 'work_type'])
    # .to_excel(r'asu_i_zu_test.xlsx')
    
# result.performer.apply(lambda w:w.last_name)

In [ ]:
# VOLS journal
result[result.system == Systems.VOLS]\
    .drop(['system', 'object', 'performer'], axis=1)\
    .reindex(columns=['date', 'place', 'work_type', 'tech_map'])\
    .sort_values(by=['date', 'place', 'work_type'])\
    .to_excel(r'vols_test.xlsx')

## email testing

In [82]:
import smtplib

In [83]:
smtp_obj = smtplib.SMTP('smtp.gmail.com', 587)

In [84]:
smtp_obj.starttls()

(220, b'2.0.0 Ready to start TLS')

In [86]:
smtp_obj.login('kzs.asu.504@gmail.com', 'kzsasu123456')

(235, b'2.7.0 Accepted')

In [ ]:
smtp_obj.sendmail('kzs.asu.504@gmail.com', 'gusev.mihail.34@gmail.com', )